In [1]:
# Import the clean tournament simulator
from clean_tournament_sim import TournamentSimulator
import pandas as pd

# Create simulator instance
sim = TournamentSimulator()
print("✅ Tournament simulator loaded!")
print("Ready to run tournaments.")


📊 Loaded 544 NA players from standings
✅ Tournament simulator loaded!
Ready to run tournaments.


In [ ]:
# Run single tournament
print("🎮 Starting single tournament...")
result = sim.run_tournament()
print("\n🏁 Tournament complete!")


In [ ]:
# Run multiple tournaments for statistical analysis
num_events = 100 # Change this number (3-5 recommended for testing)
print(f"🎮 Running {num_events} tournaments...")

multi_results = sim.run_multi_event_simulation(num_events, players_per_event=3700)  # Smaller for faster testing


In [ ]:
# 🏆 CHAMPIONSHIP POINTS TOURNAMENT + FINAL RANKINGS
# ===================================================
# This cell runs a complete tournament with real championship points
# and displays the final season standings for the top 150 players

# Create simulator with NA championship points tracking
from clean_tournament_sim import TournamentSimulator, NAStandingsManager
import pandas as pd

print("🎯 SETTING UP CHAMPIONSHIP TOURNAMENT")
print("=" * 60)

# Initialize simulator with NA standings tracking
sim = TournamentSimulator(track_na_standings=True)

if not sim.na_manager:
    print("❌ Could not load NA standings data")
    print("   Make sure 'data/na_full_data.csv' exists")
else:
    print(f"✅ Loaded {len(sim.na_manager.na_standings)} NA players")
    
    # Show championship points structure
    print(f"\n📋 Championship Points Structure:")
    sim.na_manager.show_points_structure(32)
    
    # Show pre-tournament top 10
    print(f"\n📊 CURRENT TOP 10 BEFORE TOURNAMENT:")
    print("-" * 50)
    current_top = sim.na_manager.get_top_players(10)
    for i, (_, player) in enumerate(current_top.iterrows(), 1):
        print(f"{i:2d}. {player['Top X Name'][:25]:<25} {player['Total_CP']:4d} CP")
    
    print(f"\n🎮 RUNNING CHAMPIONSHIP TOURNAMENT...")
    print("=" * 60)


In [ ]:
# Run the tournament (will automatically handle championship points)
tournament_size = 3700  # Adjust this for faster/slower execution

result = sim.run_tournament(num_players=tournament_size)

if result:
    print(f"\n🏁 TOURNAMENT COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    
    # Get the tournament results
    df = result['players_df']
    final_standings = result['final_standings']
    
    # Display tournament winner and top finishers
    print(f"\n🥇 TOURNAMENT RESULTS (Top 10):")
    print("-" * 60)
    print(f"{'Place':<5} {'Player':<25} {'CP':<5} {'Points':<6} {'NA Pts':<7}")
    print("-" * 60)
    
    for i, (_, player) in enumerate(final_standings.head(10).iterrows(), 1):
        name = player['name'][:24]
        cp = int(player['cp'])
        points = int(player['match_points'])
        
        # Check if they got championship points
        if hasattr(player, 'final_placement') and pd.notna(player['final_placement']):
            placement = int(player['final_placement'])
            na_points = sim.na_manager.get_championship_points(placement) if sim.na_manager else 0
        else:
            placement = i
            na_points = sim.na_manager.get_championship_points(placement) if sim.na_manager else 0
            
        print(f"{placement:<5} {name:<25} {cp:<5} {points:<6} {na_points:<7}")
    
    print(f"\n📊 TOURNAMENT STATISTICS:")
    print("-" * 40)
    total_players = len(df)
    day2_players = len(final_standings)
    low_cp_players = len(df[df['cp'] <= 331])
    low_cp_advancing = len(final_standings[final_standings['cp'] <= 331])
    
    print(f"Total players: {total_players:,}")
    print(f"Day 2 players: {day2_players}")
    print(f"Low CP players (<= 331): {low_cp_players:,} ({low_cp_players/total_players*100:.1f}%)")
    print(f"Low CP advancing: {low_cp_advancing} ({low_cp_advancing/day2_players*100:.1f}%)")
    
else:
    print("❌ Tournament failed to complete")


In [14]:
# 🔍 ANALYZE INDIVIDUAL PLAYER PERFORMANCE
# Use this cell to analyze specific players after running tournaments

from analyze_player_performance import analyze_player_across_tournaments, show_top_performers
import os

# Look for tournament data - check for 'complete' first, then timestamped ones
if os.path.isdir("tournament_standings_complete"):
    standings_dir = "tournament_standings_complete"
    print(f"📁 Using complete standings: {standings_dir}")
else:
    try:
        standings_dir = multi_results['standings_directory']
        print(f"📁 Tournament data found in: {standings_dir}")
    except NameError:
        print("❌ No tournament data found. Either:")
        print("   1. Run the multi-event simulation first with save_individual_standings=True, OR")
        print("   2. Make sure you have data in 'tournament_standings_complete' folder")
        standings_dir = None

if standings_dir:
    # Show top performers across all tournaments
    print(f"\n🏆 TOP PERFORMERS:")
    show_top_performers(standings_dir)
    
    # Example: Analyze a specific player
    # Replace 'Player Name' with the actual player you want to analyze
    print(f"\n🔍 EXAMPLE PLAYER ANALYSIS:")
    analyze_player_across_tournaments('Dre Alderette', standings_dir)

# To analyze any other player, use:
# analyze_player_across_tournaments('Player Name', standings_dir)


📁 Using complete standings: tournament_standings_complete

🏆 TOP PERFORMERS:
🏆 TOP 20 PERFORMERS (by average placement)
Rank Player Name               CP    Apps Avg    Best  T8s  T16s  Day2%  Consistency
----------------------------------------------------------------------------------------------------
1    Rahul Reddy               1525  83   69.2   1     6    19    100.0  0.014
2    Brent Tonisson            2020  87   73.4   1     12   19    100.0  0.016
3    Henry Chao                1738  82   73.9   2     7    18    100.0  0.014
4    Daniel Magda              1600  83   74.2   1     11   23    100.0  0.014
5    Giovanny Sasso Silva      1704  85   76.8   1     17   26    100.0  0.013
6    Owen Dalgard              1525  72   86.8   1     4    10    100.0  0.015
7    Michael Davidson          1500  76   88.2   2     10   18    100.0  0.014
8    rafael yuiti              1557  88   88.6   1     9    16    100.0  0.014
9    Christian LaBella         1228  75   90.5   1     9    18

In [12]:
# 🥊 HEAD-TO-HEAD ANALYSIS
# Analyze specific player matchups and all opponents across tournaments

from head_to_head_analyzer import analyze_head_to_head, analyze_all_opponents
import os

# Look for tournament data - check for 'complete' first, then timestamped ones
if os.path.isdir("tournament_standings_complete"):
    standings_dir = "tournament_standings_complete"
    print(f"📁 Using complete standings: {standings_dir}")
else:
    try:
        standings_dir = multi_results['standings_directory']
        print(f"📁 Using tournament data from: {standings_dir}")
    except NameError:
        print("❌ No tournament data found. Either:")
        print("   1. Run the multi-event simulation first with save_individual_standings=True, OR")
        print("   2. Make sure you have data in 'tournament_standings_complete' folder")
        standings_dir = None

if standings_dir:
    # Example 1: Head-to-head between two specific players
    print("\n" + "="*80)
    analyze_head_to_head("Isaiah C", "Dalten Cross", standings_dir)
    
    print("\n" + "="*80)
    
    # Example 2: All opponents analysis for a player
    analyze_all_opponents("Jake Santiago", standings_dir, top_n=20)

# To analyze any other matchup, use:
# analyze_head_to_head("Player A", "Player B", standings_dir)
# analyze_all_opponents("Player Name", standings_dir)


📁 Using complete standings: tournament_standings_complete

⚔️  HEAD-TO-HEAD ANALYSIS: Isaiah C vs Dalten Cross
📊 OVERALL RECORD:
   Total matches: 1
   Isaiah C: 0 wins (0.0%)
   Dalten Cross: 1 wins (100.0%)
   Ties: 0 (0.0%)

💪 CP ANALYSIS:
   Average CP difference: +171 (+ means Isaiah C higher)

🏆 TOURNAMENT PERFORMANCE (when they played):
   Isaiah C average placement: 11.0
   Dalten Cross average placement: 48.0

📋 DETAILED MATCH HISTORY:
Event   Round   Winner                   CP Diff   Brutal  Final Placements
--------------------------------------------------------------------------------
60      7       Dalten Cross             +171      No      11 vs 48

🎯 ALL OPPONENTS ANALYSIS: Jake Santiago

📊 TOP 20 MOST FACED OPPONENTS:
Opponent                 Matches   W-L-T       Win %     
------------------------------------------------------------
James C                  5         1-3-1       20.0%
Ciaran Farah             5         1-3-1       20.0%
Taylor W                 4  

In [ ]:
# 🏆 FINAL SEASON RANKINGS - TOP 150 PLAYERS
# ============================================
# Display the updated championship standings after the tournament

if sim.na_manager:
    print(f"\n🏆 FINAL NA CHAMPIONSHIP STANDINGS - TOP 150 PLAYERS")
    print("=" * 100)
    
    # Generate final rankings (automatically saves updated standings)
    final_rankings = sim.na_manager.generate_final_rankings(top_n=150)
    
    # Save to file for reference
    timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"final_rankings_top150_{timestamp}.csv"
    final_rankings.to_csv(output_file, index=False)
    
    print(f"\n💾 Top 150 rankings saved to: {output_file}")
    
    # Show some key statistics
    print(f"\n📊 CHAMPIONSHIP POINTS SUMMARY:")
    print("-" * 50)
    print(f"Total players tracked: {len(sim.na_manager.na_standings):,}")
    print(f"Players with championship points: {len(sim.na_manager.na_standings[sim.na_manager.na_standings['Total_CP'] > 0]):,}")
    print(f"Highest Total CP: {sim.na_manager.na_standings['Total_CP'].max():,}")
    print(f"Average Total CP: {sim.na_manager.na_standings['Total_CP'].mean():.1f}")
    
    # Show top 5 changes if there were any
    print(f"\n🔥 CURRENT CHAMPIONSHIP LEADERS:")
    print("-" * 50)
    top_5 = final_rankings.head(5)
    for i, (_, player) in enumerate(top_5.iterrows(), 1):
        name = player['Top X Name'][:30]
        total_cp = int(player['Total_CP'])
        top_x = int(player['Top_X_CP'])
        locals_cp = int(player['Locals CP'])
        print(f"{i}. {name:<30} | Total: {total_cp:,} | Top X: {top_x:,} | Locals: {locals_cp}")
        
    print(f"\n✅ Championship points tournament and rankings complete!")
    print(f"📁 All data saved with timestamp for future reference")
    
else:
    print("❌ Cannot display final rankings - NA standings not loaded")


In [ ]:
# 🎯 140TH PLACE CUTOFF PREDICTION - INDEPENDENT SCENARIOS 
# =========================================================
# This runs INDEPENDENT tournaments to predict the range of possible 140th place cutoffs
# Each tournament starts from the same baseline (NO cumulative effects!)

# EXPERIMENT PARAMETERS (ADJUST THESE)
num_scenarios = 10        # Number of independent scenarios to simulate
tournament_size = 1000    # Players per tournament (1000 = faster, 3700 = realistic)

print(f"🎯 PREDICTING 140TH PLACE CUTOFF RANGE")
print("=" * 60)
print(f"Parameters:")
print(f"   Independent scenarios: {num_scenarios}")
print(f"   Tournament size: {tournament_size:,} players each")
print(f"   Goal: Predict where the 140th place cutoff will be")
print(f"   Method: Each tournament starts from same baseline")

# Create simulator with championship points tracking
sim = TournamentSimulator(track_na_standings=True)

if sim.na_manager:
    # Run the independent scenario analysis
    results = sim.track_140th_place_experiment(
        num_simulations=num_scenarios,
        tournament_size=tournament_size,
        save_results=True
    )
    
    if results:
        # Extract the key data for further analysis
        place_140_values = results['raw_data']
        stats = results['statistics']
        
        print(f"\n🎯 CUTOFF PREDICTION SUMMARY")
        print("=" * 50)
        print(f"Baseline:    {stats['initial_value']:,} CP")
        print(f"Predicted Range:")
        print(f"  Lowest:    {stats['min']:,} CP")
        print(f"  Highest:   {stats['max']:,} CP")
        print(f"  Most likely: {stats['mean']:,.1f} CP")
        print(f"  Std Dev:   {stats['std']:,.1f} CP")
        
        # Calculate useful metrics for cutoff prediction
        confidence_68 = stats['std']  # ~68% of results within 1 std dev
        lower_bound = stats['mean'] - confidence_68
        upper_bound = stats['mean'] + confidence_68
        
        print(f"\n📊 CUTOFF CONFIDENCE INTERVALS:")
        print(f"  68% likely between: {lower_bound:,.0f} - {upper_bound:,.0f} CP")
        print(f"  Absolute range:     {stats['min']:,} - {stats['max']:,} CP")
        
        # Store for easy access
        cutoff_prediction = {
            'baseline': stats['initial_value'],
            'scenarios': place_140_values,
            'predicted_mean': stats['mean'],
            'predicted_min': stats['min'],
            'predicted_max': stats['max'],
            'std_dev': stats['std'],
            'confidence_68_lower': lower_bound,
            'confidence_68_upper': upper_bound
        }
        
        print(f"\n✅ Cutoff prediction complete! Data stored in 'cutoff_prediction' variable")
        print(f"💾 Detailed results saved to timestamped files")
    else:
        print("❌ Prediction failed")
else:
    print("❌ Cannot run prediction - championship points system not loaded")


In [ ]:
# 🚀 QUICK CUTOFF PREDICTOR
# ==========================
# Easy one-liners for different prediction scenarios (all INDEPENDENT)

def quick_prediction(num_scenarios, size=1000):
    """Run quick 140th place cutoff prediction with independent scenarios"""
    sim = TournamentSimulator(track_na_standings=True)
    if sim.na_manager:
        return sim.track_140th_place_experiment(num_scenarios, size, save_results=True)
    return None

# UNCOMMENT THE LINE YOU WANT TO RUN:

# Quick test (5 independent scenarios, 1000 players each)
# results_quick = quick_prediction(5, 1000)

# Standard prediction (10 independent scenarios, 1000 players each) 
# results_standard = quick_prediction(10, 1000)

# Large sample (20 independent scenarios, 1000 players each)
# results_large = quick_prediction(20, 1000)

# Realistic size (10 independent scenarios, 3700 players each) - SLOW!
results_realistic = quick_prediction(10, 3700)

print("🚀 Quick Cutoff Predictor Ready!")
print("Each scenario is INDEPENDENT - no cumulative effects")
print("Uncomment one of the lines above and run this cell")


In [ ]:
# Show single tournament summary (from last single tournament run)
sim.tournament_summary()


In [ ]:
# Analyze specific player
player_name = "Charlie Lockyer"  # Change this name

print(f"🔍 Analyzing player: {player_name}")
sim.analyze_player(player_name)


In [ ]:
# Check for successful low CP players
sim.find_successful_low_cp(8)  # 8+ wins threshold


In [ ]:
# Quick stats
if sim.tournament_result is not None:
    df = sim.tournament_result['players_df']
    final = sim.tournament_result['final_standings']
    
    # Basic stats
    total_players = len(df)
    low_cp_count = len(df[df['cp'] <= 331])
    advanced = len(df[df['match_points'] >= 19])
    low_cp_advanced = len(df[(df['match_points'] >= 19) & (df['cp'] <= 331)])
    
    # Top cut stats
    top8_low = len(final.head(8)[final.head(8)['cp'] <= 331])
    top16_low = len(final.head(16)[final.head(16)['cp'] <= 331])
    top32_low = len(final.head(32)[final.head(32)['cp'] <= 331])
    
    # Champion
    champion = final.iloc[0]
    
    print("📊 QUICK TOURNAMENT STATS")
    print("=" * 40)
    print(f"Total Players: {total_players:,}")
    print(f"Low CP (≤331): {low_cp_count:,} ({low_cp_count/total_players*100:.1f}%)")
    print(f"Advanced to Day 2: {advanced}")
    print(f"Low CP Advanced: {low_cp_advanced} ({low_cp_advanced/advanced*100:.1f}%)")
    print()
    print(f"Champion: {champion['name']} ({champion['cp']} CP)")
    print(f"Champion Record: {champion['wins']}-{champion['losses']}-{champion['ties']}")
    print()
    print(f"Low CP in Top 8: {top8_low}/8 ({top8_low/8*100:.1f}%)")
    print(f"Low CP in Top 16: {top16_low}/16 ({top16_low/16*100:.1f}%)")
    print(f"Low CP in Top 32: {top32_low}/32 ({top32_low/32*100:.1f}%)")
    
else:
    print("❌ No tournament results available. Run a tournament first!")
